In [1]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import glob
import logging
import sys

import deepposekit as dpk

import TrainingGeneratorTFRecord as TGTFR
import apt_dpk 
import run_apt_expts_2 as rae
import APT_interface as apt
import PoseTools as pt
import multiResData as mrd
import open_pose_data as opd
import util
import tfdatagen
import apt_dpk_exps as ade

import time
from os.path import expanduser

import mpl_toolkits.axes_grid1 as axg1

DPK_DSET = '/home/al/git/dpkd/datasets/fly/annotation_data_release.h5'

Using TensorFlow backend.
W0605 10:58:48.588803 140289466017600 deprecation_wrapper.py:119] From /home/al/git/APT_aldl/deepnet/open_pose4.py:12: The name tf.keras.initializers.random_normal is deprecated. Please use tf.compat.v1.keras.initializers.random_normal instead.



Your cache is: /dat0/apt/cache0426
Your models are: ['dpk']


In [2]:
defaultlogger = logging.getLogger()
defaultlogger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter("[%(asctime)s] %(levelname)s [%(name)s.%(funcName)s:%(lineno)d] %(message)s")
ch.setFormatter(formatter)
for handler in defaultlogger.handlers[:]:
    defaultlogger.removeHandler(handler)
defaultlogger.addHandler(ch)


In [3]:
dg = dpk.DataGenerator(DPK_DSET)

LEAPSTRIPPEDLBL = '/dat0/jrcmirror/groups/branson/bransonlab/apt/experiments/data/leap_dataset_gt_stripped.lbl'
EXPNAME = 'val10pct'
CACHE = '/dat0/apt/cache'
conf = apt.create_conf(LEAPSTRIPPEDLBL, 0, EXPNAME, \
                       CACHE, 'dpkfly', quiet=False)
conf.img_dim = 1  # hack, the leap stripped lbl has NumChans=3, but we created the tfr 
                  # directly using the dpk h5 which is 1-channel
conf.cachedir

/dat0/venv/env/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


'/dat0/apt/cache/leap_dset/dpkfly/view_0/val10pct'

In [4]:
conf = apt_dpk.update_conf_dpk(conf,
                               dg.graph,
                               dg.swap_index,
                               n_keypoints=dg.n_keypoints,
                               imshape=dg.compute_image_shape(),
                               useimgaug=True,
                               imgaugtype='dpkfly')
conf.batch_size = 16
conf.dpk_use_tfdata = True
conf.display_step = 84

[2020-06-05 10:58:53,916] INFO [root.update_conf_dpk:473] DPK size stuff: imsz=(192, 192), imsz_pad=(192, 192), imsz_net=(192, 192), rescale=1.0, n_trans_min=5


In [ ]:
vars(conf
    )

In [5]:
class MyCbk(tf.keras.callbacks.Callback):

    def on_train_batch_begin(self, bch, logs):
        print('otrnbb')
        print(bch)
        print(logs)
    def on_test_batch_begin(self, bch, logs):
        print('otstbb')
        print(bch)
        print(logs)
    def on_train_batch_end(self, bch, logs):
        print('otrnbe')
        print(bch)
        print(logs)
    def on_test_batch_end(self, bch, logs):
        print('otstbe')
        print(bch)
        print(logs)
    def on_epoch_begin(self, epoch, logs):
        print('oeb')
        print(epoch)
        print(logs)
    def on_epoch_end(self, epoch, logs):
        print('oee')
        print(epoch)
        print(logs)

cbks = [MyCbk()]

In [7]:
USEDG = True

if USEDG:
    VALSPLIT = 0.1
    assert conf.dpk_downsample_factor == 2
    assert conf.dpk_input_sigma == 5.0
    assert conf.dpk_graph_scale == 1.0
    tg = dpk.TrainingGenerator(generator=dg,
                               downsample_factor=conf.dpk_downsample_factor,
                               augmenter=conf.dpk_augmenter,
                               use_graph=True,
                               shuffle=True,
                               sigma=conf.dpk_input_sigma,
                               validation_split=VALSPLIT,
                               graph_scale=conf.dpk_graph_scale,
                               random_seed=0)
    sdn = dpk.models.StackedDenseNet(tg,
                                     n_stacks=conf.dpk_n_stacks,
                                     growth_rate=conf.dpk_growth_rate,
                                     pretrained=True,
                                     )
    DECAY = 0.0  # LR modulated via callback
    assert conf.dpk_base_lr_factory == .001
    optimizer = tf.keras.optimizers.Adam(
        lr=conf.dpk_base_lr_factory, beta_1=0.9, beta_2=0.999, epsilon=None,
        decay=DECAY, amsgrad=False)
    sdn.compile(optimizer=optimizer, loss='mse')
    
    VALBSIZE = 8
    EPOCHS = 10
    bsize = conf.batch_size
    sdn.fit(
        batch_size=bsize,
        validation_batch_size=VALBSIZE,
        callbacks=cbks,
        epochs=EPOCHS,
        steps_per_epoch=3,  # validation_steps=VALSTEPS,
        verbose=2
    )
else:
    tgtfr, sdn = apt_dpk.compile(conf)
    #cbks = ade.create_callbacks_exp2orig_train(conf, sdn)

    bsize = conf.batch_size
    valbsize = 16
    dstrn = sdn.train_generator(sdn.n_outputs,
                                bsize,
                                validation=False,
                                confidence=True,
                                shuffle=True,
                                infinite=True)
    dsval = sdn.train_generator(sdn.n_outputs,
                                valbsize,
                                validation=True,
                                confidence=True,
                                shuffle=False,
                                infinite=False)
    #sdn.activate_callbacks(cbks)
    

    train_model = sdn.train_model

    epochs = 100
    steps_per_epoch = 3
    train_model.fit(dstrn,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    verbose=1,
                    callbacks=cbks,
                    validation_data=dsval,
                    validation_steps=9,
                   )

                        
# Cached images in strippedlbl differ from dpk h5! 
# - Ims are 3-chan grayscale vs 1-chan
# - Locs are off-by-one; strippedlbl prob correct (0-based)
#apt.create_tfrecord(conf,split=False,use_cache=True)


# So rather than use apt.create_tfrecord, we create the tfr directly from the h5 even 
# though it may be off-by-one for replication purposes

fit_generator kwargs: {'verbose': 2}
oeb
0
{}
Epoch 1/10
otrnbb
0
{'batch': 0, 'size': 16}
otrnbe
0
{'batch': 0, 'size': 16, 'loss': 635.0797, 'output_0_loss': 210.82149, 'output_1_loss': 212.19814, 'output_2_loss': 212.06012}
otrnbb
1
{'batch': 1, 'size': 16}
otrnbe
1
{'batch': 1, 'size': 16, 'loss': 611.772, 'output_0_loss': 208.77869, 'output_1_loss': 208.41676, 'output_2_loss': 206.2304}
otrnbb
2
{'batch': 2, 'size': 16}
otrnbe
2
{'batch': 2, 'size': 16, 'loss': 601.1177, 'output_0_loss': 208.59438, 'output_1_loss': 205.83836, 'output_2_loss': 201.55705}
otstbb
0
{'batch': 0, 'size': 8}
otstbe
0
{'batch': 0, 'size': 8, 'loss': 582.6643, 'output_0_loss': 204.0145, 'output_1_loss': 194.5859, 'output_2_loss': 184.06392}
otstbb
1
{'batch': 1, 'size': 8}
otstbe
1
{'batch': 1, 'size': 8, 'loss': 596.7048, 'output_0_loss': 206.40002, 'output_1_loss': 196.90883, 'output_2_loss': 186.3757}
otstbb
2
{'batch': 2, 'size': 8}
otstbe
2
{'batch': 2, 'size': 8, 'loss': 590.941, 'output_0_loss': 20

KeyboardInterrupt: 

In [10]:
19*8

152

In [ ]:
train_model.metrics_names



In [ ]:
cbks

In [ ]:
150//16


In [ ]:
tgtfr = TGTFR.TrainingGeneratorTFRecord(conf)

In [ ]:
sdn = dpk.models.StackedDenseNet(tgtfr, 
                                 n_stacks=2, 
                                 growth_rate=48, 
                                 pretrained=True)

In [ ]:
dir(sdn.train_model)

In [ ]:
import importlib
importlib.reload(tfdatagen)

In [ ]:
dsval = tgtfr.get_tfdataset(validation=True, confidence=True, n_outputs=1,
                            shuffle=False, infinite=False,
                            instrumented=True,instrumentedname='val')

In [ ]:
tfdatagen.thecounter


In [ ]:
conf.batch_size

In [ ]:
resDSraw = tfdatagen.read_ds_idxed(dsval,range(2))

In [ ]:
len(resDSraw)


In [ ]:
resDSraw[0][0].shape

In [ ]:
tfdatagen.thecounter

In [ ]:
dir(dsval)

In [ ]:
resDSraw[0][2]

In [ ]:
imsDSraw, locsDSraw, tgtsDSraw = xylist2xyarr(resDSraw)
imsDSraw.shape, locsDSraw,shape, tgtsDSraw.shape

In [ ]:
resDS = read_ds_idxed(ds, range(150//4))

In [ ]:
imsDS, tgtsDS = xylist2xyarr(resDS)
imsDS.shape, tgtsDS.shape

In [ ]:
tgtsDS[-140,...]

In [ ]:
np.allclose(imsdpk,imsDS,), np.allclose(tgtsdpk, tgtsDS), \
np.array_equal(imsdpk, imsDS), np.array_equal(tgtsdpk, tgtsDS)

In [ ]:
tgtsDS[2,:]

In [ ]:
imsstk = np.moveaxis(imsDS, 0, -1)
imsstk = imsstk[:,:,0,:]
imsstk.shape

In [ ]:
montage(imsDS[:10,...],locs=tgtsDS[:10,...],cmap='gray',locsmrkrsz=40)

In [ ]:

tf.keras.Model.fit()

In [ ]:
cpth5 = '/dat0/apt/cache/leap_dset/dpkfly/view_0/val10pct/best_model_sdn.h5'


In [ ]:
sdn = dpk.models.load_model(cpth5, generator=None)

In [ ]:
dg = dpk.DataGenerator(DPK_DSET)
